# Para saber mais: comparação de validação cruzada para o Decision Tree Classifier

Durante o curso, abordamos a questão de que a validação cruzada não aninhada tende a exibir, em geral, um viés mais otimista em comparação com a validação aninhada. Isso ocorre porque a validação cruzada não aninhada usa os mesmos dados tanto para ajustar os parâmetros do modelo quanto para avaliar o desempenho do mesmo. No entanto, será que podemos verificar essa afirmação na prática? É exatamente isso que vamos explorar agora.

Vamos desenvolver um código capaz de calcular a pontuação de um modelo de árvore de decisão, comparando essas pontuações utilizando a validação cruzada aninhada e não aninhada. Para realizar essa tarefa, empregaremos a busca aleatória. Em seguida, apresentaremos dois gráficos comparativos. Embora haja uma quantidade considerável de código envolvido, prometo que será uma experiência interessante!

O código a seguir captura as pontuações para a validação cruzada aninhada e não aninhada do modelo Decision Tree Classifier. O conjunto de dados utilizado é o mesmo usado durante o curso, com a mesma divisão entre treino e teste.

(Prepara o café, porque o código pode demorar bastante... aqui foram 20 minutos)

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier 
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('./dados_inadimplencia.csv')
x = df.drop('inadimplente', axis=1)
y = df['inadimplente']
x_treino, X_test, y_treino, y_test = train_test_split(x, y, test_size=.33, random_state=42, stratify=y)

# Número de tentativas aleatórias
NUM_TRIALS = 30

# Definindo a grade de hiperparâmetros
p_grid = {"max_depth": [10, 20, 30, 40], 
          "min_samples_split": [2, 5, 10], 
          "min_samples_leaf": [1, 2, 4]}  

dt_classifier = DecisionTreeClassifier(random_state=42)

# Arrays para armazenar as pontuações.
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop para cada tentativa
for i in range(NUM_TRIALS):
    inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)
    outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=i)

    # Pesquisa e pontuação de parâmetros não aninhados
    clf = RandomizedSearchCV(estimator=dt_classifier, 
                             param_distributions=p_grid, 
                             cv=outer_cv, # usamos o outer_cv neste caso
                             n_iter=18, 
                             random_state=i)
    clf.fit(x_treino, y_treino)
    non_nested_scores[i] = clf.best_score_

    # CV aninhado com otimização de parâmetros
    clf = RandomizedSearchCV(estimator=dt_classifier, 
                             param_distributions=p_grid, 
                             cv=inner_cv, # inner_cv para trabalharmos com validação cruzada aninhada
                             n_iter=18, 
                             random_state=i)
    nested_score = cross_val_score(clf, X=x_treino, y=y_treino, cv=outer_cv)
    nested_scores[i] = nested_score.mean()


In [2]:
print('Scores para valização cruzada não aninhada')
non_nested_scores

Scores para valização cruzada não aninhada


array([0.67113686, 0.67676913, 0.67318562, 0.6785093 , 0.67646159,
       0.67594849, 0.6758465 , 0.67144508, 0.6818886 , 0.67912359,
       0.67666709, 0.67533554, 0.67594896, 0.67441337, 0.67543733,
       0.67666663, 0.67564217, 0.678202  , 0.67973797, 0.67789505,
       0.67472125, 0.67881626, 0.68526719, 0.67236667, 0.67380017,
       0.67810009, 0.67420946, 0.67584767, 0.67861138, 0.67779276])

In [3]:
print('Scores para valização cruzada aninhada')
nested_scores

Scores para valização cruzada aninhada


array([0.66878232, 0.67584775, 0.67226412, 0.67861159, 0.67697346,
       0.67492449, 0.674823  , 0.67001183, 0.6798416 , 0.67799755,
       0.6769738 , 0.67400437, 0.6753347 , 0.67154645, 0.67461808,
       0.67461871, 0.67635879, 0.67779263, 0.67922555, 0.67717842,
       0.67205891, 0.67615379, 0.68373177, 0.67246904, 0.67369775,
       0.67707592, 0.67298046, 0.6765643 , 0.67533496, 0.67799751])

Agora temos as pontuações da validação cruzada não aninhada e aninhada armazenadas em nested_scores e non_nested_scores, respectivamente. No entanto, precisamos de uma forma mais visual de analisar esses resultados, uma vez que atualmente temos apenas dois arrays com vários números. Para isso vamos utilizar a biblioteca Plotly.

In [4]:
import plotly.express as px

score_difference = non_nested_scores - nested_scores

# Cria um DataFrame para armazenar os escores
df = pd.DataFrame({
    'Trial': range(NUM_TRIALS),
    'Non-Nested CV Score': non_nested_scores,
    'Nested CV Score': nested_scores,
    'Score Difference': score_difference
})

# Cria um gráfico de linha para os escores de CV aninhados e não aninhados
fig = px.line(df, x='Trial', y=['Non-Nested CV Score', 'Nested CV Score'], 
              labels={'value': 'Escore', 'variable': 'Tipo de CV', 'Trial': 'Tentativa Individual #'},
              title='Validação Cruzada Não Aninhada e Aninhada')
fig.show()

# Cria um gráfico de barras para a diferença de scores
fig = px.bar(df, x='Trial', y='Score Difference',
             labels={'Score Difference': 'diferença de escores', 'Trial': 'Tentativa Individual #'},
             title='Diferença de Escores entre CV Não Aninhado e Aninhado')
fig.show()

Com estes dois gráficos podemos observar que a validação cruzada não-aninhada produz scores maiores na maioria dos casos, evidenciando uma métrica mais otimista.